<a href="https://colab.research.google.com/github/aaronbannin/colab-clones/blob/main/Advent_of_Haystack_Pipeline_Connecting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advent of Haystack: Day 1
_Make a copy of this Colab to start!_

In this first challenge, we are going to build a RAG pipeline that answers questions based on the contents of a URL. Most of the pipeline is ready, but your task is to complete the pipeline connections 👇

You should complete Step 5 of this colab.

### Components to use:
1. `LinkContentFetcher`: Expects a list of URLs and returns `ByteStream` type
2. `HTMLToDocument`: Expects a `ByteStream` and creates `Document` type.
3. `DocumentSplitter`: This expcects a list of `Documents` and returns a list of split, preprocessed documents.
4. `PromptBuilder`: To define the manner we want to interact with an LLM. We use Jinja templating
5. `GPTGenereator`: Expects a fully formed prompt and queries an OpenAI GPT model.

### 1) Installation

**Note:** There is a known issue with colab due to a version conflict error related to `llmx` which comes with Colab. You might get an `llmx` error. You can safely ignore this, or run `pip uninstall -y llmx`

In [ ]:
!pip install haystack-ai
!pip install boilerpy3

### 2) Enter API keys for LLM and search providers
Run this code and you’ll be prompted to enter your OpenAI API Key. If you don’t have a key, [follow these instructions](https://help.openai.com/en/articles/4936850-where-do-i-find-my-api-key).

In [ ]:
import getpass
import os

openai_api_key = getpass.getpass("Enter OpenAI API key:")

### 3) Create components

In [ ]:
from haystack.components.fetchers import LinkContentFetcher
from haystack.components.converters import HTMLToDocument
from haystack.components.preprocessors import DocumentSplitter
from haystack.components.builders.prompt_builder import PromptBuilder
from haystack.components.generators import GPTGenerator

fetcher = LinkContentFetcher()
converter = HTMLToDocument()
splitter = DocumentSplitter(split_length=100, split_overlap=5)

In [ ]:
template = """Given the information below: \n
            {% for document in documents %}
                {{ document.content }}
            {% endfor %}
            Question: {{ query }}. \n Answer:"""

prompt_builder = PromptBuilder(template = template)

In [ ]:
llm = GPTGenerator(api_key = openai_api_key, model_name = "gpt-4")

### 4) Add them to a Haystack 2.0 Pipeline

In [ ]:
from haystack import Pipeline

pipeline = Pipeline()
pipeline.add_component(name="fetcher", instance=fetcher)
pipeline.add_component(name="converter", instance=converter)
pipeline.add_component(name="splitter", instance=splitter)
pipeline.add_component(name="prompt_builder", instance=prompt_builder)
pipeline.add_component(name="llm", instance=llm)

###5) Connect the components

Complete the pipelne connections to achieve a working pipeline that can be run

**PSA:** If you are re-running this cell multiple times and you get a `PipelineConnectionError`, try restarting your Colab runtime.

In [ ]:
pipeline.connect("fetcher", "converter")

###6) Run the Pipeline

In [ ]:
query_dict ={
    "urls": ["https://haystack.deepset.ai/blog/customizing-rag-to-summarize-hacker-news-posts-with-haystack2"],
    "query": "How do you build a custom component?"
}


result = pipeline.run(data={"fetcher": {"urls": query_dict["urls"]}, "prompt_builder": {"query": query_dict["query"]}})

In [ ]:
print(result['llm']['replies'][0])

###7) Draw the Pipeline 🎨
When you run this code block, it will create a new file that will appear in the "Files" section of Colab (see menu tab on the side).

In [ ]:
pipeline.draw("/content/pipeline.png")